# Jackknife

Suponga que se tiene una muestra de variables independientes e indénticamente distribuidas $\mathbf{x}=(x_1,\ldots,x_n)$ que provienen de una distribución $F$ que no se conoce. 

Utilizando $\mathbf{x}$, se estima una cantidad de interés $\theta$ a través de un algoritmo $A$.

$$
\hat{\theta} = A(\mathbf{x})
$$

Si la distribución $F$ fuera conocida, se podría estimar el error estándar de $\hat{\theta}$ generando nuevas muestras $\mathbf{x}$ obteniendo un conjunto $\hat{\theta_1}, \hat{\theta_2}, \ldots, \hat{\theta_m}$ de estimaciones de $\theta$, el error estándar se calcularía obteniendo la desviación estándar de estas estimaciones.

Al no conocer $F$, la metodología de **jackknife** propone estimar el error estándar de la siguiente forma:

Sea $\mathbf{x_{(i)}} = (x_1, \ldots, x_{i-1},x_{i+1},\ldots,x_n)$ la muestra $\mathbf{x}$ sin incluir la $i$-ésima observación. Si $\hat{\theta_{(i)}} = A(\mathbf{x_{(i)}})$ denota el valor estimado de nuestra cantidad de interés, entonces el estimado del erro estándar está dado por

$$
\widehat{se}_{jack} = \sqrt{ \dfrac{n-1}{n} \sum_{i=1}^{n} \left(\hat{\theta_{(i)}} - \hat{\theta_{(*)}}   \right)^2  }
$$

en donde $\hat{\theta_{(*)}} = \sum_{i=1}^{n} \dfrac{\hat{\theta_{(i)}}}{n}$

**Ejercicio**

Simule 60 variables $N(\mu=2,\sigma=1)$ y programe una función para calcular el error estándar, utilizando jackknife, al estimar $\mu$ a través del promedio.

Compare este error con la desviación estándar realizando el procedimiento descrito si se conociera la distribución $F$. Utilice $1e4$ simulaciones.


**Observación**

A pesar de que en este ejercicio se considera que $\mathbf{x}$ es una muestra de números reales, la metodología funciona también para otro tipo de objetos, por ejemplo muestras de vectores en $\mathbb{R}^2$.

In [1]:
import numpy as np
from scipy.stats import norm


def algoritmo(x):
    '''
    Algoritmo para estimar un parámetro a partir
    de una muestra
    
    ENTRADA
    x: Numpy array
    
    SALIDA
    Estimación de nuestro parámetro de interés
    '''
    return np.mean(x)

def jackkife_se(x, algoritmo):
    '''
    Función para calcular el error estándar utilizando
    la metodología de jackknife
    
    ENTRADA
    x: Numpy array 1D que representa la muestra
    
    algoritmo: Función que calcula el parámetro de
    interés
    
    SALIDA
    estimados: numpy array con el parámetro estimado para cada muestra
    se: Estimación del error estándar para el parámetro
    de interés
    '''
    
    #Número de obervaciones
    n_obs = len(x)
    
    #Para almacenar los estimados al quitar x_i
    estimados = []
    
    for i in range(n_obs):
        
        #Quita la observación x_i
        muestra = np.concatenate((x[0:i], x[i+1:]))
        
        #calcula el estimado
        estimados.append(algoritmo(muestra))
    
    #calcula el error estándar estimado
    se = np.sqrt((n_obs - 1)*np.var(estimados, ddof = 0))
    
    return np.array(estimados), se

In [2]:
np.random.seed(54321)
size_muestra = 60
#muestra para jackknife
x_jack = norm.rvs(size = size_muestra, loc = 2, scale = 1)

#Error estándar estimado
estimados_jack, se_jack = jackkife_se(x_jack, algoritmo)

#Ahora simulamos como si se conociera F
x_F = norm.rvs(size = (int(1e4), size_muestra), loc = 2, scale = 1)

estimados_F = np.apply_along_axis(algoritmo, axis = 1, arr = x_F)
se_F = np.std(estimados_F,ddof=0)

print('El error estándar de jackknife es', np.round(se_jack,5))
print('El error estándar conociendo F es', np.round(se_F,5))

El error estándar de jackknife es 0.11438
El error estándar conociendo F es 0.12945


# Bootstrap

El estimado del error estándar utilizando la metodología de bootstrap se basa en, a partir de una muestra inicial $\mathbf{x}=(x_1,x_2,\ldots,x_n)$ obtener nuevas muestras

$$
\mathbf{x}^{*} = (x_{1}^{*},x_{2}^{*},\ldots,x_{n}^{*})
$$

en donde cada $x_{i}^{*}$ se obtiene a través de una extracción equiprobable y con reemplazo del conjunto $\{ x_1, x_2, \ldots, x_n\}$.

Para cada muestra $\mathbf{x}^{*}$ obtenemos un estimado $\hat{\theta^{*b}}$ de nuestro parámetro de interés.

Si $B$ denota el número de muestras $\mathbf{x^*}$, el estimado para el error estándar es

$$
\hat{se}_{boot} = \sqrt{ \dfrac{\sum_{i=1}^{B} \left(\hat{\theta^{*b}} - \hat{\theta^{*}} \right)^2}{B-1} }
$$

en donde 
$$
\hat{\theta^{*}} = \dfrac{1}{B}\sum_{i=1}^{B} \hat{\theta^{*b}}
$$

**Ejercicio**

Simule 60 variables $N(\mu=2,\sigma=1)$ y programe una función para calcular el error estándar, utilizando bootstrap, al estimar $\mu$ a través del promedio.

Compare este error con la desviación estándar realizando el procedimiento descrito si se conociera la distribución $F$. Utilice $1e4$ simulaciones.

$B=1000$


In [3]:
import numpy as np
from scipy.stats import norm


def algoritmo(x):
    '''
    Algoritmo para estimar un parámetro a partir
    de una muestra
    
    ENTRADA
    x: Numpy array
    
    SALIDA
    Estimación de nuestro parámetro de interés
    '''
    return np.mean(x)

def bootstrap_se(x, algoritmo, B = 1000):
    '''
    Función para calcular el error estándar utilizando
    la metodología de bootstrap
    
    ENTRADA
    x: Numpy array 1D que representa la muestra original
    
    algoritmo: Función que calcula el parámetro de
    interés
    
    B: Entero positivo que representa el número de
    muestras bootstrap
    
    SALIDA
    estimados: numpy array con el parámetro estimado para cada muestra
    se: Estimación del error estándar para el parámetro
    de interés
   '''
    #Para almacenar los estimados de cada muestra
    estimados = []
    
    for i in range(B):
        
        #Obtiene muestra con reemplazo
        muestra = np.random.choice(a = x, size = len(x), replace = True)

        #calcula el estimado
        estimados.append(algoritmo(muestra))
    
    #calcula el error estándar estimado
    se = np.std(estimados,ddof=1)
    
    return np.array(estimados), se

In [4]:
np.random.seed(54321)
boot_muestra = 70
#muestra para jackknife
x_boot = norm.rvs(size = size_muestra, loc = 2, scale = 1)

#Error estándar estimado
estimados_boot, se_boot = bootstrap_se(x_boot, algoritmo, B = boot_muestra)

#Ahora simulamos como si se conociera F
x_F = norm.rvs(size = (int(1e4), boot_muestra), loc = 2, scale = 1)

estimados_F = np.apply_along_axis(algoritmo, axis = 1, arr = x_F)
se_F = np.std(estimados_F,ddof=0)

print('El error estándar de bootstrap es', np.round(se_boot,5))
print('El error estándar conociendo F es', np.round(se_F,5))

El error estándar de bootstrap es 0.11473
El error estándar conociendo F es 0.12013


In [5]:
#Construcción del intervalo del 95%
(np.quantile(estimados_boot, q = 0.025), np.quantile(estimados_boot, q = 0.975) )

(2.026573076310199, 2.4529342639891354)

In [7]:
estimados_boot.mean()

2.2194100448841416